In [1]:
"""This module contains functions to scrape all the activities listed on a city page."""
import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import openpyxl

In [2]:
def get_city_url_list(city_url, max_pages=100):
    """The city url is the first page of activities, most city urls have multiple pages
    of activities, so we need to check if the page exists and if it does, add it to
    a dictionary of valid pages. The key is the page number and the value is the url.
    The variable max_pages is the maximum number of pages to check, this is to prevent
    the scraper from running for too long if the city has a lot of pages."""

    response = requests.get(city_url, timeout=5, allow_redirects=False)
    if response.status_code != 200:
        print(f"City url is not valid: {city_url}")
        return {}
    print(f"Checked validity for page 1 and it was 200")

    valid_pages = {1: response}
    if max_pages > 1:
        for page_number in range(2, max_pages+1):
            start = time.time()
            page_url = city_url + f"/{page_number}"
            response = requests.get(page_url, timeout=5)
            if response.status_code == 200:
                valid_pages[page_number] = response
            print(f"Checked validity for page {page_number} and it was {response.status_code}. Returned in {time.time() - start} seconds")
    print(f"Found {len(valid_pages)} valid pages")
    return valid_pages

In [3]:
def get_activity_urls(soup):
    """This function takes bs4 soup object of a city activity page and returns a list of
    activity urls contained in it."""

    activity_urls = []
    activity_links = soup.find_all("a", class_='text-dark highlight-able card-link')
    for link in activity_links:
        activity_urls.append("https://viator.com" + link['href'])
    return activity_urls

In [4]:
def extract_data(link):
    """This function takes a link to an activity page and returns a list
    of the data contained in it."""

    data = []
    response = requests.get(link, timeout=5)
    soup = bs(response.text, 'html.parser')


    activity_name = soup.find("h1", class_="title__1Wwg title2__C3R7").text

    activity_url = link

    price_per_participant = soup.find_all("span", class_="moneyView__2HPx defaultColor__1NL9")[0].text

    valid_number_of_participants = soup.find("input", class_ = "input__MNXR md__1Fp3")['data-automation-value']

    valid_date = soup.find("input", class_ = "input__2pmO md__2bZz")["value"]

    try:
      overview_block = soup.find("div", class_="overviewWrapper__bMs4")
      main_overview = overview_block.find_all("div")[0]
      activity_description = main_overview.find_all("div")[0].text
    except:
      activity_description = "Timed out while waiting for description. Please check link for description."
      print(activity_description)

    details_block = soup.find_all("div", class_ = "item__3eVq")
    details = [x.find_all("div")[1].text for x in details_block]
    duration = ""
    for detail in details:
        if any(chr.isdigit() for chr in detail):
          duration = detail

    try:
      avg_rating = soup.find("span", class_ = "averageRatingValue__Q1ep").text
      review_count = soup.find("div", class_ = "reviewCount__3sJa").text
    except:
      avg_rating = "No rating available"
      review_count = "No review available"

    

    
    data.append(activity_name)
    data.append(activity_url)
    data.append(price_per_participant)
    data.append(valid_number_of_participants)
    data.append(valid_date)
    data.append(activity_description)
    data.append(duration)
    data.append(avg_rating)
    data.append(review_count)

    return data

    

In [5]:
def main_scraper(city_url, max_pages=100, dataframe=pd.DataFrame(columns=['Activity Name', 'Activity URL', 'Price per Participant', 'Valid Number of Participants', 'Valid Date', 'Activity Description', 'Duration', 'Average Rating', 'No. of Reviews'])):
    """This function takes a city url and returns a dataframe of all the activities
    listed on the city page."""

    valid_pages = get_city_url_list(city_url, max_pages)
    activity_urls = []
    for page_number, response in valid_pages.items():
        start = time.time()
        soup = bs(response.text, 'html.parser')
        activity_urls += get_activity_urls(soup)
        print(f"Activity urls for page {page_number} added to list in {time.time() - start} seconds")
    print(f"Found {len(activity_urls)} activity urls")

    print(f"Estimated time to completion: {len(activity_urls) * 1.8} seconds")

    activity_len = len(activity_urls)
    for link in activity_urls:
        start = time.time()
        try:
            data = extract_data(link)
            dataframe.loc[len(dataframe)] = data
            activity_len -= 1
            print(f"Extracted data for {link} in {time.time() - start} seconds. {activity_len} activities left")
        except Exception as e:
            print(f"Error extracting data for {link} because {e}")
            continue
    print(f"Finished scraping {city_url}")
    return dataframe
    

In [ ]:
if __name__ == "__main__":
    df = pd.DataFrame(columns=['Activity Name', 'Activity URL', 'Price per Participant', 'Valid Number of Participants', 'Valid Date', 'Activity Description', 'Duration', 'Average Rating', 'No. of Reviews'])
    df = main_scraper("https://www.viator.com/Bangkok/d343-ttd", max_pages = 84)
    df.to_excel("bangkok_activities.xlsx", index=False)